##### **Setup**
##### Download report from BCC
1. Go to the web bank client
2. Under each currency account choose "Выписка"
3. After choosing a period -> "Отправить на почту" 

##### Download report from CAIXA
1. Go to the web bank client
2. Current accounts -> Balances and Operations
3. Choose dates and download csv
4. Delete header and footer


#### to-do
- add flag for not using in analysis
- check categories per month
- add analitics for cashback (and when is is added to the account)
- top-10 biggest spendings in every category
- check subscriptions
- separate preparation of data with analytics

In [ ]:
#imports
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

In [ ]:
#parse bcc euro table
df_euro = pd.read_html('euro_bcc_export.html')[2]
#parse bcc tenge table
df_tenge = pd.read_html('tenge_bcc_export.html')[2]
# parse caixa euro table
df_euro_caixa = pd.read_csv('euro_caixa_export.csv', sep=";") 

### Preparing BCC Euro DF

In [ ]:
#renaming headers
df_euro = df_euro.rename(
    columns={
        '№ п/п': 'id',
        'Дата': 'record_dt',
        'Дебет': 'sum',
        'Кредит': 'sum_temp',
        'Назначение': 'details', }
)

#changing types
df_euro['id'] = df_euro['id'].astype('int')

df_euro['sum'] = df_euro['sum'].str.replace(r'\s+', '', regex=True)
df_euro['sum'] = pd.to_numeric(df_euro['sum'])

df_euro['sum_temp'] = df_euro['sum_temp'].str.replace(r'\s+', '', regex=True)
df_euro['sum_temp'] = pd.to_numeric(df_euro['sum_temp'])

df_euro['record_dt'] = pd.to_datetime(df_euro['record_dt'], format='%d.%m.%Y')

In [ ]:
#moving all sums to one column
def move_income_to_sum_column(row):
    if pd.isna(row['sum']):
        row['sum'] = row['sum_temp']
    if pd.isna(row['sum_temp']):
        row['sum'] = -row['sum']
    return row

df_euro = df_euro.apply(move_income_to_sum_column, axis=1)
df_euro = df_euro.drop('sum_temp', axis=1)

In [ ]:
#parsing details
first_word = r'(^\w+)'
forex = r'^Покупка иностранной валюты'
atm = r'^Снятие наличных АТМ'
transfer = r'^Перевод \(списание\)'
returns = r'^Прочие зачисления на карту \(credit\)'
retail_regex = r'^(?:[^,]*,){4}([^,]*)'
retail_loc_regex = r'^[^,]*,[^,]*,\s*([^,]*,[^,]*)'
retail_datetime_regex = r'^[^,]*,\s*(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'

def check_retail(row):
    if re.match(first_word, row['details']).group(0) == 'Retail':
        row['pos_loc'] = re.search(retail_loc_regex, row['details']).group(1)
        row['pos'] = re.search(retail_regex, row['details']).group(1)
        row['transaction_dt'] = re.search(retail_datetime_regex, row['details']).group(1)
        row['category_1'] = 'retail'
    elif re.match(forex, row['details']):
        row['category_1'] = 'forex'
    elif re.match(atm, row['details']):
        row['category_1'] = 'atm'
    elif re.match(transfer, row['details']):
        row['category_1'] = 'transfer'
    elif re.match(returns, row['details']):
        row['category_1'] = 'returns'
    else:
        row['category_1'] = 'unallocated'
    return row

df_euro = df_euro.apply(check_retail, axis=1)

In [ ]:
# Regular expressions to extract date and retail sum directly in the loop
cashback_date_regex = r'Дата (\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'
retail_sum_regex = r'сумма ([\d.]+)'

# Step 2: Track indices of cashback rows that have matching retail transactions
matched_cashback_indices = []

# Step 3: Process cashback rows, find matching Retail rows, and add cashback sum
for index, row in df_euro.iterrows():
    if "Учет вознаграждений по CashBack" in row['details']:
        # Extract the cashback date and retail transaction sum directly
        cashback_date_match = re.search(cashback_date_regex, row['details'])
        retail_sum_match = re.search(retail_sum_regex, row['details'])
        
        if cashback_date_match and retail_sum_match:
            cashback_date = cashback_date_match.group(1)
            retail_amount = float(retail_sum_match.group(1))
            cashback_amount = row['sum']
            
            # Find matching Retail row by 'transaction_dt' and 'sum'
            matching_retail_index = df_euro[(df_euro['transaction_dt'] == cashback_date) & 
                                            (df_euro['sum'] == -retail_amount)].index
            
            # If match found, update cashback_sum in the Retail row and mark cashback row for deletion
            if not matching_retail_index.empty:
                df_euro.loc[matching_retail_index, 'cashback_sum'] = cashback_amount
                matched_cashback_indices.append(index)

# Step 4: Remove matched cashback rows
df_euro.drop(matched_cashback_indices, inplace=True)

# Reset index after dropping rows (optional)
df_euro.reset_index(drop=True, inplace=True)

### Preparing BCC Tenge DF

In [ ]:
#renaming headers
df_tenge.columns = [' '.join(col).strip() for col in df_tenge.columns.values]

df_tenge = df_tenge.rename(
    columns={
        df_tenge.columns[0]: 'record_dt',
        df_tenge.columns[1]: 'transaction_dt',
        df_tenge.columns[2]: 'details',
        df_tenge.columns[3]: 'sum_in_currency',
        df_tenge.columns[4]: 'currency',
        df_tenge.columns[5]: 'fee',
        df_tenge.columns[6]: 'sum',
        df_tenge.columns[7]: 'cashback',
    }
    
)
df_tenge.head(10)

df_tenge['sum_in_currency'] = df_tenge['sum_in_currency'].str.replace(r'\s+', '', regex=True)
df_tenge['sum_in_currency'] = pd.to_numeric(df_tenge['sum_in_currency'])

df_tenge['sum'] = df_tenge['sum'].str.replace(r'\s+', '', regex=True)
df_tenge['sum'] = pd.to_numeric(df_tenge['sum'])

df_tenge['fee'] = df_tenge['fee'].str.replace(r'\s+', '', regex=True)
df_tenge['fee'] = pd.to_numeric(df_tenge['fee'])

df_tenge['cashback'] = df_tenge['cashback'].str.replace(r'\s+', '', regex=True)
df_tenge['cashback'] = pd.to_numeric(df_tenge['cashback'])

df_tenge['record_dt'] = pd.to_datetime(df_tenge['record_dt'], format='%d.%m.%Y')
df_tenge['transaction_dt'] = pd.to_datetime(df_tenge['transaction_dt'], format='%d.%m.%Y')

In [ ]:
#parsing details
first_word = r'(^\w+)'
forex = r'^Покупка иностранной валюты'
cashback = r'^Перевод кешбэка на карту'
transfer_to_me1 = r'^Перевод с карты'
transfer_to_me2 = r'^Пополнение карт.счета'
salary1 = r'^Пополнение от ТОО Яндекс.Казахстан'
salary2 = r'^Пополнение от TOO "Aim High Technology"'
retail_regex = r'^(?:[^,]*,){4}([^,]*)'
retail_loc_regex = r'^[^,]*,[^,]*,\s*([^,]*,[^,]*)'
retail_datetime_regex = r'^[^,]*,\s*(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'

def category_1_parsing(row):
    if re.match(first_word, row['details']).group(0) == 'Retail':
        row['pos_loc'] = re.search(retail_loc_regex, row['details']).group(1)
        row['pos'] = re.search(retail_regex, row['details']).group(1)
        row['transaction_dt'] = re.search(retail_datetime_regex, row['details']).group(1)
        row['category_1'] = 'retail'
    elif re.match(forex, row['details']):
        row['category_1'] = 'forex'
    elif re.match(transfer_to_me1, row['details']) or re.match(transfer_to_me2, row['details']):
        row['category_1'] = 'transfer_to_me'
    elif re.match(cashback, row['details']):
        row['category_1'] = 'cashback'
    elif re.match(salary1, row['details']) or re.match(salary2, row['details']):
        row['category_1'] = 'salary'
    else:
        row['category_1'] = 'unallocated'
    return row

df_tenge = df_tenge.apply(category_1_parsing, axis=1)

### Prepare Caixa Euro DF

In [ ]:
# Renaming headers and changing types
df_euro_caixa = df_euro_caixa.rename(
    columns={
        df_euro_caixa.columns[0]: 'record_dt',
        df_euro_caixa.columns[1]: 'transaction_dt',
        df_euro_caixa.columns[2]: 'details',
        df_euro_caixa.columns[3]: 'sum',
        df_euro_caixa.columns[4]: 'sum_temp',
    }
    
)


df_euro_caixa["sum_temp"] = df_euro_caixa["sum_temp"].str.replace(",", "", regex=False).astype(float)
df_euro_caixa['sum_temp'] = pd.to_numeric(df_euro_caixa['sum_temp'])


df_euro_caixa["sum"] = df_euro_caixa["sum"].str.replace(",", "", regex=False).astype(float)
df_euro_caixa['sum'] = pd.to_numeric(df_euro_caixa['sum'])

df_euro_caixa['record_dt'] = pd.to_datetime(df_euro_caixa['record_dt'], format='%d-%m-%Y')
df_euro_caixa['transaction_dt'] = pd.to_datetime(df_euro_caixa['transaction_dt'], format='%d-%m-%Y')

In [ ]:
#moving all sums to one column
def move_income_to_sum_column(row):
    if pd.isna(row['sum']):
        row['sum'] = row['sum_temp']
    if pd.isna(row['sum_temp']):
        row['sum'] = -row['sum']
    return row

df_euro_caixa = df_euro_caixa.apply(move_income_to_sum_column, axis=1)
df_euro_caixa = df_euro_caixa.drop('sum_temp', axis=1)

In [ ]:
#parsing details
first_word = r'(^\w+)'
retail = r'^COMPRAS'
mbway_transfer = r'^TRF MBWAY'
deposit = r'^DEPOSITO'
transfer_from_bcc = r'^TRF P2P GERMAN'
insurance = r'^Fidelidad'
bills_payment = r'^PAGAMENTO'



def category_1_parsing(row):
    if re.match(retail, row['details']):
        row['category_1'] = 'retail'
    elif re.match(mbway_transfer, row['details']):
        row['category_1'] = 'mbway_transfer'
    elif re.match(deposit, row['details']):
        row['category_1'] = 'deposit'
    elif re.match(transfer_from_bcc, row['details']):
        row['category_1'] = 'transfer_from_bcc'
    elif re.match(insurance, row['details']):
        row['category_1'] = 'insurance'
    elif re.match(bills_payment, row['details']):
        row['category_1'] = 'bills_payment'
    else:
        row['category_1'] = 'unallocated'
    return row

df_euro_caixa = df_euro_caixa.apply(category_1_parsing, axis=1)

### Merging and Mapping

In [ ]:
df_euro["source"] = "bcc_euro"
df_tenge["source"] = "bcc_tenge"
df_euro_caixa["source"] = "caixa_euro"

df_merged = pd.concat([df_euro, df_tenge, df_euro_caixa], axis=0, join="outer", ignore_index=True)


In [ ]:
# Load excel file with category mapping
df_cat_mapping = pd.read_excel("category_mapping.xlsx", sheet_name="Sheet1")

cat_mapping = {}
for _, row in df_cat_mapping.iterrows():
    cat = row['category']
    pos = row['pos']
    if cat not in cat_mapping:
        cat_mapping[cat] = []
    cat_mapping[cat].append(pos)

#def assign_category(row):
 #   for category, keywords in cat_mapping.items():
  #      if any(keyword in row['details'] for keyword in keywords):
   #         return category
    #return 'unallocated'
def assign_category(row):
    best_category = 'unallocated'
    best_keyword_len = 0
    
    for category, keywords in cat_mapping.items():
        for keyword in keywords:
            if keyword in row["details"]:
                if len(keyword) > best_keyword_len:
                    best_keyword_len = len(keyword)
                    best_category = category
                    
    return best_category

df_merged["category_2"] = df_merged.apply(assign_category, axis=1)
df_merged["pos"] = df_merged["pos"].fillna(df_merged["details"])

### Reports

In [ ]:
df_merged.to_excel("report.xlsx", index=False)

In [ ]:
#df_euro[df_euro['category_2'] == 'clothes'].sort_values(by='sum').head(10)

monthly_sum = df_merged.groupby([df_merged['record_dt'].dt.to_period('M'), 'category_2'])['sum'].sum().reset_index()

# Pivot the DataFrame to have months as columns
pivot_table = monthly_sum.pivot(index='category_2', columns='record_dt', values='sum').fillna(0)

# Convert the PeriodIndex to a string for clarity
pivot_table.columns = pivot_table.columns.astype(str)

# Display the pivot table
pivot_table

In [ ]:
# Make a file with uncallocated to manually add to mapping
df_cat1_unallocated = df_merged[df_merged["category_1"] == "unallocated"]
df_cat1_unallocated = df_cat1_unallocated[["record_dt", "transaction_dt", "details", "sum", 'pos']]

df_cat2_unallocated = df_merged[df_merged["category_2"] == "unallocated"]
df_cat2_unallocated = df_cat2_unallocated[["record_dt", "transaction_dt",'category_1', "details", "sum",'pos']]


#df_unallocated.to_excel("unallocated.xlsx", index=False)
with pd.ExcelWriter("unallocated.xlsx") as writer:
    df_cat1_unallocated.to_excel(writer, sheet_name="Sheet1", index=False)
    df_cat2_unallocated.to_excel(writer, sheet_name="Sheet2", index=False)